# OpenMP complex compute constructs

In the previous section's exercises on simple single line compute constructs, we made extensive use of the combined compute directive `#pragma omp target teams distribute parallel for`.
At the time, we discussed briefly what each clause in this directive was doing, and in this notebook we will explore what happens if we break apart this instruction and combine the clauses in different ways.
We will then investigate how different approaches to parallelising nested loops impacts the performance of the code.

Let us begin by checking that we have an appropriate GPU to run our code on using the `rocm-smi`, moving into our work directory and making sure that our work environment is clean.

In [ ]:
rocm-smi
cd $HOME/DiRAC-AMD-GPU/notebooks/02-OpenMP/2c-OpenMP_complex_constructs/C
make clean

We want the operating system to be able to automatically handle the movement of data and control between the host and the device, so we will set the `HSA_XNACK` variable to 1.
For these exercises, we want to see what the implementation is doing for each compute directive.
We can do this by setting the `LIBOMPTARGET_KERNEL_TRACE` environment to 1.
As the name suggests, this will print a trace for each OpenMP kernel encountered during runtime, and will give us information on the details of the execution.
We will see this in action soon - for now, let's set those variables.

In [ ]:
export HSA_XNACK=1
export LIBOMPTARGET_KERNEL_TRACE=1

## Breaking down compute constructs

Firstly, let's return to the full directive.
The code in [`saxpy_gpu_parallelfor.c`](./C/saxpy_gpu_parallelfor.c) is the same as that in the previous notebook - that is to say, a simple saxpy code offloaded to the GPU using the `#pragma omp target teams distribute parallel for` directive.

Let's build and run it, and then we can break down the outputs:

In [ ]:
make saxpy_gpu_parallelfor
./saxpy_gpu_parallelfor

There is a lot of information here.
Let's break down the important outputs here:
 - **DEVID** - the ID of the device  the kernel is running on,
 - **teamsXthrds** - the number of teams generated, and how many threads are within each team,
 - **sgpr_count** - the number of scalar registers required by a wavefront,
 - **vgpr_count** - the number of vector registers required by each work-item.

For the full combined directive, we see that 416 teams are generated with 256 threads each.
There is a low vector register usage of 8.
Bear in mind also the time this kernel took to run - which should be of order 0.075 seconds - as we will use this as a comparison metric as well.

### The target directive

Let's start by looking at the `target` directive itself.
In the example code [`saxpy_gpu_target.c`](./C/saxpy_gpu_target.c), we use the target directive with no additional clauses.
Let's see this directive as it is in the code:

In [ ]:
grep "pragma" saxpy_gpu_target.c

The target directive instructs the compiler to offload the enclosed code to the device.
In our example, this is the subsequent saxpy `for` loop.
With no further instructions or tips given to the compiler on what the code might be doing or how we might want it to run, let's find out how well it can handle the loop for itself:

In [ ]:
make saxpy_gpu_target
./saxpy_gpu_target

Whilst we have successfully offloaded the code the target device, we can see that we have only created a single working group of 256 threads.
In this configuration, there is little point to using the GPU at all, a fact that is borne out by a run time almost two orders of magnitude larger than our original baseline run.
We are effectively running the code in serial.

But of course without adding in any parallelisation instructions to the `target` directive, we should have expected this.
Let's look at what happens if we now spawn a number of parallel processes, but give them no further directions.

### The teams clause

In the example code [`saxpy_gpu_target_teams.c`](./C/saxpy_gpu_target_teams.c), we have added the `teams` clause to the `target` directive.
Let's see the current pragma now:

In [ ]:
grep "pragma" saxpy_gpu_target_teams.c

The `teams` clause instructs the pragma to create a "league of teams".
The initial thread of each team executes the code region with all of the data on each thread.
Let's see how the target teams instruction handles our loop:

In [ ]:
make saxpy_gpu_target_teams
./saxpy_gpu_target_teams

We've successfully offloaded the code, and created a set of 624 teams to run the code, but we've still ended up with a significantly longer run time.
So what is going on here?

The answer is that although we've created a set of teams to run our loop, and they are indeed doing so in parallel.
But without any instructions that the loop can be split up in any way, each team ends up running the entire loop itself!

You may also notice that the results we get out from the loop are not the expected values of `y[0] 4.000000` and `y[N-1] 4.000000`.
This is because each team is running using the same arrays, with no atomic protection.
This leads to race conditions and array elements being added and written in ways in which they were not intended.

Let's look now at letting the teams share the work properly.

### The distribute clause

The code [`saxpy_gpu_target_teams_distribute.c`](./C/saxpy_gpu_target_teams_distribute.c) adds the `distribute` clause to the `target` directive. The pragma now looks like:

In [ ]:
grep "pragma" saxpy_gpu_target_teams_distribute.c

The `distribute` clause splits up loop iterations across available teams, and executes them on the main thread.
This should solve both problems we had in the previous example by properly atomising the running of the code.
Let's see it in action now:

In [ ]:
make saxpy_gpu_target_teams_distribute
./saxpy_gpu_target_teams_distribute

Success!
Even without the `parallel for` clause, we are now approaching the same speed as our baseline with the full directive.
Interestingly, we have more teams in this example than the baseline - 624 compared to 416 - but not all threads are being used.

Try changing the array size in the example, or replacing the kernel with something with more work, and see how the run time and load changes.

### Parallel for without a teams distribute clause

As a final experiment, let's see what happens if we include the `parallel for` clause in our `target` directive without the `teams distribute` clauses.
The example [`saxpy_gpu_target_parallel_for.c`](./C/saxpy_gpu_target_parallel_for.c) has this already set up for us.

In [ ]:
make saxpy_gpu_target_parallel_for
./saxpy_gpu_target_parallel_for

As expected, no additional teams have been created, but we still see a run time approaching that of the original baseline case.
The `parallel` clause creates multiple threads within a team, and the `for` clause spreads out the work between these threads.
So even without a high teams count, the `parallel for` clause can give very good performance.
This can be used to run efficiently with fewer GPU compute units.

## Exploring the split level directives

Finally in this notebook, let's look at different approaches to offloading nested `for` loops to the GPU.

[`saxpy_gpu_collapse.c`](./C/saxpy_gpu_collapse.c) contains a nested `for` loop parallelised using the `collapse` directive discussed in the previous notebook.
The code of interest is the loop itself:

```C
#pragma omp target teams distribute parallel for collapse(2)
for (int j = 0; j < N; j++) {
   for (int i = 0; i < M; i++) {
      y[j][i] = a * x[j][i] + y[j][i];
   }
}
```

As we previously discussed, the `collapse(2)` clause asks the compiler to collapse the 2 subsequent loops into a single one that can be distributed out amongst the GPU computer.

But now that we've tested the various clauses of this directive, can we not think of another way of splitting this code on the GPU?
Could we not envisage each iteration of the first loop running its own second loop independently, hopefully also in parallel?

The example code [`saxpy_gpu_split_level.c`](./C/saxpy_gpu_split_level.c) does exactly that in its revised version of the nest `for` loops:

```C
#pragma omp target teams distribute   
for (int j = 0; j < N; j++) {         
   #pragma omp parallel for simd      
   for (int i = 0; i < M; i++) {      
      y[j][i] = a * x[j][i] + y[j][i];
   }                                  
}                                     
```

Here we create a set of teams and distribute the first loop amongst them.
Then the second loop is parallelised within each team.

Which approach do you think will be faster?

Let's run them both now and compare.

In [ ]:
make saxpy_gpu_collapse
./saxpy_gpu_collapse
make saxpy_gpu_split_level
./saxpy_gpu_split_level

There is almost nothing to chose between the two in terms of run time, at least in their original configuration.
We do see, however, that the collapse statement largely increases the number of teams created in the `target` directive - 3907 compared to 624.
We are using the GPU compute resources efficiently with the split level approach.

Try changing the size of the input arrays, and seeing how this impacts performance.

You should notice that at larger sizes (say `N=10000,M=10000`), the split level directive begins running faster, whilst also keeping at a lower number of teams created.

Now that we've explored the `target` directive and some of its many possible clauses and configurations, let's move on to memory management directives.
In the next section, we will at explicit memory directive.